In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型對於電鋸資料集的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]

#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    train_loss_TL = []
    train_accuracy_TL = []
    val_loss_TL = []
    val_accuracy_TL = []
    test1_accuracy_TL = []
    test3_accuracy_TL = []
    test5_accuracy_TL = []
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    
    #資料抓入
    test_loader1 = epg.getTestDataset(sound[0])
    test_loader3 = epg.getTestDataset(sound[1])
    test_loader5 = epg.getTestDataset(sound[2])
    
    for TL in range(2,-1,-1):
        print("----------------------------------------------------------------------------------------------------------------")
        print("TL: ", TL+1, "fold: ", fold)
        print("----------------------------------------------------------------------------------------------------------------")
        #資料抓入
        train, val = epg.getTrainDataset(fold, sound[TL], num_class)
        #分批方法
        train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
        val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
        
        #優化器
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        #調度器
        scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        #訓練模型
        train_loss, train_accuracy, val_loss, val_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[TL], num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5)
    
        #將gpu資料取回
        train_loss = torch.stack(train_loss,dim=-1).cpu()
        train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
        val_loss = torch.stack(val_loss,dim=-1).cpu()
        val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
        for i in range(num_epochs):
            # 寫入資料
            f.write("TL :{} fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(TL+1 ,fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        train_loss_TL.append(train_loss)
        train_accuracy_TL.append(train_accuracy)
        val_loss_TL.append(val_loss)
        val_accuracy_TL.append(val_accuracy)
        test1_accuracy_TL.append(test1_accuracy)
        test3_accuracy_TL.append(test3_accuracy)
        test5_accuracy_TL.append(test5_accuracy)
    modelName = "Fold_" + str(fold) + "_TF.pkl"
    torch.save(model, modelName)
    train_loss_array.append(train_loss_TL)
    train_accuracy_array.append(train_accuracy_TL)
    val_loss_array.append(val_loss_TL)
    val_accuracy_array.append(val_accuracy_TL)
    test1_accuracy_array.append(test1_accuracy_TL)
    test3_accuracy_array.append(test3_accuracy_TL)
    test5_accuracy_array.append(test5_accuracy_TL)
f.close()

----------------------------------------------------------------------------------------------------------------
TL:  3 fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.9911187887191772 Train_acc: 26.875000% Val_Loss: 1.862892508506775 Val_accuracy: 41.250000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 2/30 Train_Loss: 1.705538272857666 Train_acc: 50.312500% Val_Loss: 1.7781239748001099 Val_accuracy: 55.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 3/30 Train_Loss: 1.5337979793548584 Train_acc: 58.750000% Val_Loss: 1.6308255195617676 Val_accuracy: 60.000000%
Test accuracy of the model: 39.000000%
Test accuracy of the model: 48.000000%
Test accuracy of the model: 56.000000%
Train Epoch: 4/30 Train_Loss: 1.43523812

Train Epoch: 3/30 Train_Loss: 0.6747452020645142 Train_acc: 83.958336% Val_Loss: 0.983404815196991 Val_accuracy: 74.166672%
Test accuracy of the model: 83.000000%
Test accuracy of the model: 93.000000%
Test accuracy of the model: 96.000000%
Train Epoch: 4/30 Train_Loss: 0.6581047773361206 Train_acc: 84.791672% Val_Loss: 0.6362296342849731 Val_accuracy: 70.416672%
Test accuracy of the model: 94.800000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 5/30 Train_Loss: 0.42624011635780334 Train_acc: 86.041672% Val_Loss: 0.7518763542175293 Val_accuracy: 72.500000%
Test accuracy of the model: 93.000000%
Test accuracy of the model: 95.333333%
Test accuracy of the model: 98.000000%
Train Epoch: 6/30 Train_Loss: 0.4754747450351715 Train_acc: 87.187508% Val_Loss: 0.7133233547210693 Val_accuracy: 72.916672%
Test accuracy of the model: 93.200000%
Test accuracy of the model: 96.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 7/30 Train_Loss: 0.

Train Epoch: 6/30 Train_Loss: 0.43380844593048096 Train_acc: 88.062500% Val_Loss: 1.2927501201629639 Val_accuracy: 71.250000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 7/30 Train_Loss: 0.33676788210868835 Train_acc: 88.375000% Val_Loss: 1.714815378189087 Val_accuracy: 70.250000%
Test accuracy of the model: 95.600000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 8/30 Train_Loss: 0.2942224442958832 Train_acc: 89.062500% Val_Loss: 0.9068301320075989 Val_accuracy: 70.750000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 9/30 Train_Loss: 0.3371139168739319 Train_acc: 89.250000% Val_Loss: 0.5387625098228455 Val_accuracy: 69.750000%
Test accuracy of the model: 96.600000%
Test accuracy of the model: 98.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 10/30 Train_Loss

Test accuracy of the model: 35.000000%
Test accuracy of the model: 40.000000%
Train Epoch: 9/30 Train_Loss: 1.055259346961975 Train_acc: 76.250000% Val_Loss: 1.5451548099517822 Val_accuracy: 73.750000%
Test accuracy of the model: 72.200000%
Test accuracy of the model: 85.666667%
Test accuracy of the model: 94.000000%
Train Epoch: 10/30 Train_Loss: 0.9953661561012268 Train_acc: 79.062500% Val_Loss: 1.2752681970596313 Val_accuracy: 71.250000%
Test accuracy of the model: 48.400000%
Test accuracy of the model: 58.000000%
Test accuracy of the model: 71.000000%
Train Epoch: 11/30 Train_Loss: 0.9668625593185425 Train_acc: 82.812500% Val_Loss: 1.1849756240844727 Val_accuracy: 72.500000%
Test accuracy of the model: 56.400000%
Test accuracy of the model: 76.666667%
Test accuracy of the model: 86.000000%
Train Epoch: 12/30 Train_Loss: 0.9238388538360596 Train_acc: 82.187500% Val_Loss: 1.4882975816726685 Val_accuracy: 75.000000%
Test accuracy of the model: 46.200000%
Test accuracy of the model: 60

Test accuracy of the model: 95.200000%
Test accuracy of the model: 98.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 12/30 Train_Loss: 0.35862839221954346 Train_acc: 90.520836% Val_Loss: 0.5500108003616333 Val_accuracy: 87.916672%
Test accuracy of the model: 93.800000%
Test accuracy of the model: 97.333333%
Test accuracy of the model: 98.000000%
Train Epoch: 13/30 Train_Loss: 0.48530805110931396 Train_acc: 91.458336% Val_Loss: 0.46999070048332214 Val_accuracy: 86.666672%
Test accuracy of the model: 95.800000%
Test accuracy of the model: 98.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 14/30 Train_Loss: 0.3088690936565399 Train_acc: 91.666672% Val_Loss: 0.4671306610107422 Val_accuracy: 85.416672%
Test accuracy of the model: 94.600000%
Test accuracy of the model: 98.000000%
Test accuracy of the model: 98.000000%
Train Epoch: 15/30 Train_Loss: 0.24862222373485565 Train_acc: 92.604172% Val_Loss: 0.7357410788536072 Val_accuracy: 85.416672%
Test accuracy of the mod

...

Test accuracy of the model: 91.800000%
Test accuracy of the model: 95.333333%
Test accuracy of the model: 98.000000%
Train Epoch: 27/30 Train_Loss: 0.6473824977874756 Train_acc: 87.187500% Val_Loss: 0.7474287748336792 Val_accuracy: 83.750000%
Test accuracy of the model: 91.800000%
Test accuracy of the model: 95.333333%
Test accuracy of the model: 98.000000%
Train Epoch: 28/30 Train_Loss: 0.7629297971725464 Train_acc: 86.250000% Val_Loss: 0.8434348702430725 Val_accuracy: 86.250000%
Test accuracy of the model: 93.000000%
Test accuracy of the model: 95.666667%
Test accuracy of the model: 98.000000%
Train Epoch: 29/30 Train_Loss: 0.6156147718429565 Train_acc: 86.875000% Val_Loss: 0.6513747572898865 Val_accuracy: 86.250000%
Test accuracy of the model: 90.800000%
Test accuracy of the model: 95.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 30/30 Train_Loss: 0.7277555465698242 Train_acc: 86.875000% Val_Loss: 0.7013676166534424 Val_accuracy: 86.250000%
Test accuracy of the model: 

Test accuracy of the model: 98.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 30/30 Train_Loss: 0.16754022240638733 Train_acc: 95.729172% Val_Loss: 0.40032604336738586 Val_accuracy: 91.666672%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  4
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 0.43837010860443115 Train_acc: 83.625000% Val_Loss: 0.7802391648292542 Val_accuracy: 80.750000%
Test accuracy of the model: 97.400000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 2/30 Train_Loss: 0.4349260926246643 Train_acc: 85.937500% Val_Loss: 0.685422956943512 Val_accuracy: 78.000000%
Test accuracy of the model:

Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 2/30 Train_Loss: 1.7166749238967896 Train_acc: 43.750000% Val_Loss: 1.935141682624817 Val_accuracy: 51.250000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Test accuracy of the model: 0.000000%
Train Epoch: 3/30 Train_Loss: 1.4920549392700195 Train_acc: 51.250000% Val_Loss: 1.549093246459961 Val_accuracy: 53.750000%
Test accuracy of the model: 41.200000%
Test accuracy of the model: 50.000000%
Test accuracy of the model: 52.000000%
Train Epoch: 4/30 Train_Loss: 1.48540198802948 Train_acc: 54.375000% Val_Loss: 1.4232912063598633 Val_accuracy: 62.500000%
Test accuracy of the model: 75.000000%
Test accuracy of the model: 87.000000%
Test accuracy of the model: 94.000000%
Train Epoch: 5/30 Train_Loss: 1.3416305780410767 Train_acc: 54.062500% Val_Loss: 1.6660661697387695 Val_accuracy: 61.250000%
Test accuracy of the model: 72.600000%
Tes

Test accuracy of the model: 96.000000%
Test accuracy of the model: 97.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 5/30 Train_Loss: 0.6268976330757141 Train_acc: 86.354172% Val_Loss: 0.7268034815788269 Val_accuracy: 80.416672%
Test accuracy of the model: 95.800000%
Test accuracy of the model: 98.666667%
Test accuracy of the model: 99.000000%
Train Epoch: 6/30 Train_Loss: 0.43724268674850464 Train_acc: 87.708336% Val_Loss: 0.5807917714118958 Val_accuracy: 75.416672%
Test accuracy of the model: 93.200000%
Test accuracy of the model: 97.333333%
Test accuracy of the model: 98.000000%
Train Epoch: 7/30 Train_Loss: 0.6682707667350769 Train_acc: 87.291672% Val_Loss: 0.7141957879066467 Val_accuracy: 80.000008%
Test accuracy of the model: 96.200000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 8/30 Train_Loss: 0.4261719882488251 Train_acc: 87.395836% Val_Loss: 0.7949562668800354 Val_accuracy: 80.000008%
Test accuracy of the model: 96.

Test accuracy of the model: 97.400000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 8/30 Train_Loss: 0.2893661856651306 Train_acc: 89.375000% Val_Loss: 0.8321990966796875 Val_accuracy: 70.750000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 9/30 Train_Loss: 0.33693361282348633 Train_acc: 88.750000% Val_Loss: 1.66822350025177 Val_accuracy: 73.500000%
Test accuracy of the model: 97.200000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 10/30 Train_Loss: 0.3003055453300476 Train_acc: 88.000000% Val_Loss: 1.3470590114593506 Val_accuracy: 73.000000%
Test accuracy of the model: 96.600000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 11/30 Train_Loss: 0.2723185122013092 Train_acc: 90.875000% Val_Loss: 0.6214383840560913 Val_accuracy: 71.750000%
Test accuracy of the model: 

In [4]:
#設定存檔位置
f = open("lastData.txt", "w")
for TL in range(3):
    train_loss_last = []
    train_accuracy_last = []
    val_loss_last = []
    val_accuracy_last = []
    test1_accuracy_last = []
    test3_accuracy_last = []
    test5_accuracy_last = []
    
    #整理出訓練完的數值
    for fold in range(5):
        train_loss_last.append(train_loss_array[fold][TL][num_epochs-1])
        train_accuracy_last.append(train_accuracy_array[fold][TL][num_epochs-1])
        val_loss_last.append(val_loss_array[fold][TL][num_epochs-1])
        val_accuracy_last.append(val_accuracy_array[fold][TL][num_epochs-1])
        test1_accuracy_last.append(test1_accuracy_array[fold][TL][num_epochs-1])
        test3_accuracy_last.append(test3_accuracy_array[fold][TL][num_epochs-1])
        test5_accuracy_last.append(test5_accuracy_array[fold][TL][num_epochs-1])
        # 寫入資料
        f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][TL][num_epochs-1], train_accuracy_array[fold][TL][num_epochs-1], val_loss_array[fold][TL][num_epochs-1], val_accuracy_array[fold][TL][num_epochs-1], test1_accuracy_array[fold][TL][num_epochs-1], test3_accuracy_array[fold][TL][num_epochs-1], test5_accuracy_array[fold][TL][num_epochs-1]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    # 寫入資料
    f.write("Max Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(val_loss_last), max(val_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
    f.write("Min Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(val_loss_last), min(val_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
    f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(val_loss_last), np.mean(val_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
f.close()